## Install any missing packages

In [ ]:
!nvidia-smi

In [ ]:
!pip install segmentation-models-pytorch


In [ ]:
import os
import numpy as np
import pandas as pd
from glob import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision.models as models


# Step 1: Load BUSI Dataset

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aryashah2k/breast-ultrasound-images-dataset")

print("Path to dataset files:", path)

#Check dataset 

import os

DATA_PATH = "/kaggle/input/breast-ultrasound-images-dataset"  # adjust if needed

# Check files
for root, dirs, files in os.walk(DATA_PATH):
    if files:
        print(root, "->", len(files), "files")

##  1: Collect Images and Masks

In [ ]:
import os
from glob import glob

DATA_PATH = "/kaggle/input/breast-ultrasound-images-dataset/Dataset_BUSI_with_GT/"

# Find all images and masks
all_files = glob(os.path.join(DATA_PATH, "**/*.png"), recursive=True)

# Separate images and masks
images = [f for f in all_files if "mask" not in f.lower()]
masks = [f for f in all_files if "mask" in f.lower() or "gt" in f.lower()]

print(f"Total images: {len(images)}")
print(f"Total masks: {len(masks)}")


## Match Images to Masks

In [ ]:
# Map each image to its mask based on filename
image_to_mask = {}
for img_path in images:
    base = os.path.basename(img_path).split('.')[0]
    mask_match = [m for m in masks if base in os.path.basename(m)]
    if mask_match:
        image_to_mask[img_path] = mask_match[0]
    else:
        image_to_mask[img_path] = None  # fallback if mask missing

print(f"Images with masks: {sum([m is not None for m in image_to_mask.values()])}")


## Train/val split

In [ ]:
from sklearn.model_selection import train_test_split

# Keep only images with masks for segmentation
segmentation_imgs = [img for img, m in image_to_mask.items() if m is not None]
segmentation_masks = [image_to_mask[img] for img in segmentation_imgs]

train_imgs, val_imgs, train_masks, val_masks = train_test_split(
    segmentation_imgs, segmentation_masks, test_size=0.2, random_state=42
)

print("Train images:", len(train_imgs), "Validation images:", len(val_imgs))


## U-Net Dataset Class

In [ ]:
import torch
from torch.utils.data import Dataset
import cv2
import numpy as np

class UltrasoundDataset(Dataset):
    def __init__(self, images, masks, img_size=(256, 256), transform=None):
        self.images = images
        self.masks = masks
        self.img_size = img_size
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Load grayscale
        img = cv2.imread(self.images[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)

        # Resize
        img = cv2.resize(img, self.img_size)
        mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)

        # Normalize
        img = img / 255.0
        mask = mask.astype(np.float32) / 255.0

        # Add channel dimension
        img = np.expand_dims(img, axis=0)
        mask = np.expand_dims(mask, axis=0)

        if self.transform:
            img = self.transform(img)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)


## U-Net Model (PyTorch)

In [ ]:
import torch.nn as nn
import torch

class UNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=1):
        super(UNet, self).__init__()

        def conv_block(in_c, out_c):
            return nn.Sequential(
                nn.Conv2d(in_c, out_c, 3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(out_c, out_c, 3, padding=1),
                nn.ReLU(inplace=True)
            )

        # Encoder
        self.enc1 = conv_block(in_channels, 64)
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = conv_block(64, 128)
        self.pool2 = nn.MaxPool2d(2)
        self.enc3 = conv_block(128, 256)
        self.pool3 = nn.MaxPool2d(2)

        # Bottleneck
        self.bottleneck = conv_block(256, 512)

        # Decoder
        self.up3 = nn.ConvTranspose2d(512, 256, kernel_size=2, stride=2)
        self.dec3 = conv_block(512, 256)
        self.up2 = nn.ConvTranspose2d(256, 128, kernel_size=2, stride=2)
        self.dec2 = conv_block(256, 128)
        self.up1 = nn.ConvTranspose2d(128, 64, kernel_size=2, stride=2)
        self.dec1 = conv_block(128, 64)

        # Final output
        self.final = nn.Conv2d(64, out_channels, kernel_size=1)

    def forward(self, x):
        # Encoder path
        e1 = self.enc1(x)
        e2 = self.enc2(self.pool1(e1))
        e3 = self.enc3(self.pool2(e2))

        # Bottleneck
        b = self.bottleneck(self.pool3(e3))

        # Decoder path
        d3 = self.dec3(torch.cat([self.up3(b), e3], dim=1))
        d2 = self.dec2(torch.cat([self.up2(d3), e2], dim=1))
        d1 = self.dec1(torch.cat([self.up1(d2), e1], dim=1))

        # Output: segmentation mask probability
        return torch.sigmoid(self.final(d1))


## Prepare DataLoaders

In [ ]:
import cv2
import numpy as np
import torch
from torch.utils.data import Dataset

class UltrasoundDataset(Dataset):
    def __init__(self, images, masks, img_size=(256, 256)):
        self.images = images
        self.masks = masks
        self.img_size = img_size

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        # Read grayscale image
        img = cv2.imread(self.images[idx], cv2.IMREAD_GRAYSCALE)
        mask = cv2.imread(self.masks[idx], cv2.IMREAD_GRAYSCALE)

        # Resize both image and mask to fixed size
        img = cv2.resize(img, self.img_size)             # bilinear default
        mask = cv2.resize(mask, self.img_size, interpolation=cv2.INTER_NEAREST)

        # Normalize
        img = img / 255.0
        mask = mask / 255.0
        mask = mask.astype(np.float32)

        # Add channel dimension (C,H,W)
        img = np.expand_dims(img, axis=0)
        mask = np.expand_dims(mask, axis=0)

        return torch.tensor(img, dtype=torch.float32), torch.tensor(mask, dtype=torch.float32)


In [ ]:
from torch.utils.data import DataLoader

# Use the updated UltrasoundDataset with img_size parameter
train_dataset = UltrasoundDataset(train_imgs, train_masks, img_size=(256, 256))
val_dataset   = UltrasoundDataset(val_imgs, val_masks, img_size=(256, 256))

train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=4)


## Train U-Net

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = UNet().to(device)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(10):  # start with small epochs
    model.train()
    train_loss = 0
    for imgs, masks in train_loader:
        imgs, masks = imgs.to(device), masks.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, masks)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {train_loss/len(train_loader):.4f}")
